In [1]:
import mitsuba as mi 
from matplotlib import pyplot as plt
mi.set_variant("cuda_ad_rgb")
import drjit as dr
from PDE3D.BoundaryShape import *
from PDE3D import PATH
from PDE3D.utils import *
from PDE3D.Solver import *


In [2]:
vis_conf = 0
resolution = [256,256]
spp = 512
radius = 1
slice_offset = .2
slice_axis = "z"
boundary_value = 0
def source_generator(points, params):
    return dr.select(dr.norm(points) < params["radius"], params["power"], 0)
source_p = {}
source_p["radius"] = 0.5
source_p["power"] = 1
f = FunctionCoefficient("source", source_p, source_generator)
σ = ConstantCoefficient("screening", 10)
b = ConstantCoefficient("dirichlet", boundary_value)

In [3]:
@dr.syntax
def correct_solution(points, params, radius, boundary_value):
    rc = params["radius"]
    rc2 = dr.square(rc)
    rc3 = rc2 * rc
    R = radius
    f = params["power"]
    c2 = -f * rc3 / (3 * R) + boundary_value
    c1 = f *  rc2 / 2  + c2

    r = dr.norm(points)
    r2 = dr.square(r)
    result = mi.Float(0)
    if r<rc:
        result = c1 - f * r2 / 6
    elif r<R:
        result = f * rc3 / (3 * r) + c2
    else:
        result = mi.Float(0)
    return result

In [4]:
shape = Sphere(radius = radius, dirichlet = [b])
data_holder = DataHolder(shape = shape, f=f, σ=σ, default_majorant=10.0)

In [5]:
wos = WosConstant(data_holder)
slice = Slice(offset = slice_offset, axis = slice_axis)

points, film_points = slice.create_slice_points(resolution, spp)
L, p = wos.solve(points_in= points, conf_numbers= [mi.UInt32(0), mi.UInt32(1)])

In [6]:
L_correct = correct_solution(points, source_p, radius, boundary_value)

In [7]:
image, _ =create_slice_from_result(L, resolution)
image_correct, _ =create_slice_from_result(L_correct, resolution)

In [8]:
cmap= "viridis"
cam_origin = mi.Point3f([-1.0,1.0,2])
image3D, norm1 = shape.visualize(slice = slice, slice_image = mi.TensorXf(image[vis_conf]), colormap = cmap, cam_origin= cam_origin, image_res=[512, 512])
image3D_correct, norm2 = shape.visualize(slice = slice, slice_image = mi.TensorXf(image_correct[vis_conf]), colormap = cmap, cam_origin= cam_origin, image_res=[512, 512])
image3D_diff, norm_diff = shape.visualize(slice = slice, slice_image = mi.TensorXf(image_correct[vis_conf] - image[vis_conf]), colormap = "coolwarm", 
                                          cam_origin= cam_origin, image_res=[512, 512], sym_colorbar=True)
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize = (15,5))
plot_image_3D(image3D, ax1, norm = norm1, cmap = cmap)
plot_image_3D(image3D_correct, ax2, norm = norm2, cmap = cmap)
plot_image_3D(image3D_diff, ax3, norm = norm_diff, cmap = "coolwarm")
ax1.set_title("Simulation")
ax2.set_title("Analytic")
ax3.set_title("Difference")

nanobind: implicit conversion from type 'Point3f' to type 'mitsuba.cuda_ad_rgb.ScalarPoint3f' failed!


TypeError: look_at(): incompatible function arguments. The following argument types are supported:
    1. look_at(self, origin: mitsuba.cuda_ad_rgb.ScalarPoint3f, target: mitsuba.cuda_ad_rgb.ScalarPoint3f, up: mitsuba.cuda_ad_rgb.ScalarPoint3f) -> mitsuba.cuda_ad_rgb.ScalarTransform4f

Invoked with types: mitsuba.cuda_ad_rgb.ScalarTransform4f, kwargs = { origin: mitsuba.cuda_ad_rgb.Point3f, target: mitsuba.cuda_ad_rgb.ScalarPoint3f, up: mitsuba.cuda_ad_rgb.ScalarPoint3f }